# Quantitative Value Strategy
We are going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': ' SATLK QTGBALL AEG/RCNOENSAMD)(SE',
 'calculationPrice': 'close',
 'open': 139.26,
 'openTime': 1670794411352,
 'openSource': 'filofica',
 'close': 145.88,
 'closeTime': 1644946860165,
 'closeSource': 'offacili',
 'high': 145.88,
 'highTime': 1652821839623,
 'highSource': ' erulem d5pdnya eeict1i',
 'low': 139.26,
 'lowTime': 1648194252445,
 'lowSource': ' umtcial5eeprdidn e1 ey',
 'latestPrice': 143.16,
 'latestSource': 'Close',
 'latestTime': 'July 2, 2021',
 'latestUpdate': 1700234449853,
 'latestVolume': 82072636,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 145.05,
 'delayedPriceTime': 1656125217342,
 'oddLotDelayedPrice': 142.73,
 'oddLotDelayedPriceTime': 1650320814036,
 'extendedPrice': 141.69,
 'extendedChange': -0.11,
 'extendedChangePercent': -0.00081,
 'extendedPriceTime': 1659209037381,
 'previousClose': 143.13,
 'previousVolume': 53491672,
 'change': 2.76

# Using the Batch API calls in order to reduce the time elapsed

In [4]:
def create_batch(x,n):
    for i in range(0,len(x),n):
        yield x[i:i+n]

In [5]:
batch_list = list(create_batch(stocks['Ticker'], 100))
batch_strings = []
for i in range(0, len(batch_list)):
    batch_strings.append(','.join(batch_list[i]))

df_columns = ['Ticker', 'Price', 'Price-to-earnings ratio', 'Number of Shares to Buy']

In [6]:

df = pd.DataFrame(columns=df_columns)

for batch in batch_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={batch}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    
    # print(data)

    for stock in batch.split(','):
        df = df.append(
                        pd.Series(
                            [
                                stock, 
                                data[stock]['quote']['latestPrice'],
                                data[stock]['quote']['peRatio'],
                                'N/A'
                            ], index = df_columns
                        ), ignore_index = True
        )

In [7]:
df

,Ticker,Price,Price-to-earnings ratio,Number of Shares to Buy
0,A,155.39,51.88,N/A
1,AAL,22.45,-1.51,N/A
2,AAP,215.61,22.76,N/A
3,AAPL,143.21,32.05,N/A
4,ABBV,120.84,40.91,N/A
...,...,...,...,...
500,YUM,120.35,31.75,N/A
501,ZBH,165.41,59.9,N/A
502,ZBRA,553.07,46.56,N/A
503,ZION,55.83,6.58,N/A


# Removing Glamour Stocks
Glamour stocks are stocks which are not value stocks; i.e. we are sorting the data based on our <strong>Price-to-earnings ratio</strong> and filter out top 50 of these stocks. <br> 
Glamour stocks are shares that investors believe will increase in value faster than the rest of the market. Investors also call them <strong>hot stocks</strong> They rise faster than the rest even in a bull market. The financial press views glamour stocks favorably. Buyers like them because they have very high price-earnings ratios.

In [8]:
df.sort_values('Price-to-earnings ratio',ascending=False,inplace=True)
df

,Ticker,Price,Price-to-earnings ratio,Number of Shares to Buy
109,COP,65.31,3712.8,N/A
42,APA,22.50,1047.87,N/A
160,EOG,88.46,808.49,N/A
339,NOW,570.05,761.04,N/A
206,GPC,131.93,372.4,N/A
...,...,...,...,...
363,PEG,61.50,None,N/A
442,TROW,207.19,None,N/A
452,UA,19.36,None,N/A
454,UAL,53.13,None,N/A


In [9]:
df.reset_index(drop=True,inplace=True)

In [10]:
df = df[:51]

In [11]:
df

,Ticker,Price,Price-to-earnings ratio,Number of Shares to Buy
0,COP,65.31,3712.8,N/A
1,APA,22.50,1047.87,N/A
2,EOG,88.46,808.49,N/A
3,NOW,570.05,761.04,N/A
4,GPC,131.93,372.4,N/A
5,BSX,44.33,317.99,N/A
6,GILD,69.90,299.96,N/A
7,SBAC,330.50,268.49,N/A
8,UDR,50.06,266.17,N/A
9,TDG,685.77,243.83,N/A


# Creating the Portfolio Size

In [12]:
def create_portfolio_size():
    global portfolio_size
    
    portfolio_size = input(' Please enter your portfolio amount : ')

    try:
        val = float(portfolio_size)
    except ValueError:
        portfolio_size = input("Please enter your portfolio amount : ")
    
    return portfolio_size

In [13]:
position_size = float(create_portfolio_size()) / len(df.index)

# Calculating Number of Stocks to Buy

In [14]:
for i in range(0, len(df['Ticker'])):
    total_share = position_size/df['Price'][i]
    df.loc[i,'Number of Shares to Buy'] = math.floor(total_share)

C:\Users\Preet\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
df

,Ticker,Price,Price-to-earnings ratio,Number of Shares to Buy
0,COP,65.31,3712.8,306
1,APA,22.50,1047.87,888
2,EOG,88.46,808.49,226
3,NOW,570.05,761.04,35
4,GPC,131.93,372.4,151
5,BSX,44.33,317.99,451
6,GILD,69.90,299.96,286
7,SBAC,330.50,268.49,60
8,UDR,50.06,266.17,399
9,TDG,685.77,243.83,29


# Building a Better (and More Realistic) Value Strategy

The price-to-earnings ratio doesn't work well with stocks with negative earnings.
<br>
Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.
<br>

We will filter for stocks with the lowest percentiles on the following metrics:

1. Price-to-earnings ratio
2. Price-to-book ratio
3. Price-to-sales ratio
4. Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
5. Enterprise Value divided by Gross Profit (EV/GP)

In [16]:
real_df_columns = [
    'Ticker',
    'Price',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'Value Score',
    'Number of Shares to Buy'
]

In [17]:
real_df = pd.DataFrame(columns=real_df_columns)

In [18]:
for batch in batch_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={batch}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for stock in batch.split(','):
        enterprise_value = data[stock]['advanced-stats']['enterpriseValue']
        ebitda = data[stock]['advanced-stats']['EBITDA']
        gross_profit = data[stock]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        real_df = real_df.append(
            pd.Series(
                [
                    stock,
                    data[stock]['quote']['latestPrice'],
                    data[stock]['quote']['peRatio'],
                    'N/A',
                    data[stock]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[stock]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A',
                    'N/A'
                ],index = real_df_columns
            ),ignore_index = True
        )

In [19]:
real_df

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Value Score,Number of Shares to Buy
0,A,153.44,51.32,N/A,9.81,N/A,8.12,N/A,31.870553,N/A,14.969202,N/A,N/A,N/A
1,AAL,22.54,-1.51,N/A,-1.75,N/A,1.09,N/A,-3.815853,N/A,3.181297,N/A,N/A,N/A
2,AAP,212.61,23.28,N/A,4.05,N/A,1.35,N/A,11.825002,N/A,2.826381,N/A,N/A,N/A
3,AAPL,143.14,31.53,N/A,34.29,N/A,7.2,N/A,23.126911,N/A,18.107521,N/A,N/A,N/A
4,ABBV,120.26,39.97,N/A,15.53,N/A,4.24,N/A,12.772195,N/A,8.731831,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.90,32.14,N/A,-4.5,N/A,6.06,N/A,24.326242,N/A,10.532166,N/A,N/A,N/A
501,ZBH,168.96,62.6,N/A,2.79,N/A,5.05,N/A,28.928507,N/A,8.344987,N/A,N/A,N/A
502,ZBRA,542.44,47.35,N/A,12.38,N/A,6.12,N/A,32.418297,N/A,13.697288,N/A,N/A,N/A
503,ZION,53.42,6.6,N/A,1.19,N/A,2.52,N/A,5.238580,N/A,2.558296,N/A,N/A,N/A


In [20]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    real_df[column].fillna(real_df[column].mean(), inplace = True)

In [21]:
real_df

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Value Score,Number of Shares to Buy
0,A,153.44,51.32,N/A,9.81,N/A,8.12,N/A,31.870553,N/A,14.969202,N/A,N/A,N/A
1,AAL,22.54,-1.51,N/A,-1.75,N/A,1.09,N/A,-3.815853,N/A,3.181297,N/A,N/A,N/A
2,AAP,212.61,23.28,N/A,4.05,N/A,1.35,N/A,11.825002,N/A,2.826381,N/A,N/A,N/A
3,AAPL,143.14,31.53,N/A,34.29,N/A,7.20,N/A,23.126911,N/A,18.107521,N/A,N/A,N/A
4,ABBV,120.26,39.97,N/A,15.53,N/A,4.24,N/A,12.772195,N/A,8.731831,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.90,32.14,N/A,-4.50,N/A,6.06,N/A,24.326242,N/A,10.532166,N/A,N/A,N/A
501,ZBH,168.96,62.60,N/A,2.79,N/A,5.05,N/A,28.928507,N/A,8.344987,N/A,N/A,N/A
502,ZBRA,542.44,47.35,N/A,12.38,N/A,6.12,N/A,32.418297,N/A,13.697288,N/A,N/A,N/A
503,ZION,53.42,6.60,N/A,1.19,N/A,2.52,N/A,5.238580,N/A,2.558296,N/A,N/A,N/A


In [22]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for i in real_df.index:
    for metric in metrics.keys():
        real_df.loc[i, metrics[metric]] = stats.percentileofscore(real_df[metric], real_df.loc[i, metric])/100

In [23]:
real_df

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Value Score,Number of Shares to Buy
0,A,153.44,51.32,0.823762,9.81,0.79604,8.12,0.817822,31.870553,0.829703,14.969202,0.809901,N/A,N/A
1,AAL,22.54,-1.51,0.118812,-1.75,0.083168,1.09,0.10297,-3.815853,0.037624,3.181297,0.142574,N/A,N/A
2,AAP,212.61,23.28,0.429703,4.05,0.530693,1.35,0.161386,11.825002,0.29703,2.826381,0.118812,N/A,N/A
3,AAPL,143.14,31.53,0.574257,34.29,0.958416,7.20,0.775248,23.126911,0.681188,18.107521,0.881188,N/A,N/A
4,ABBV,120.26,39.97,0.731683,15.53,0.873267,4.24,0.557426,12.772195,0.348515,8.731831,0.510891,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.90,32.14,0.586139,-4.50,0.081188,6.06,0.691089,24.326242,0.710891,10.532166,0.661386,N/A,N/A
501,ZBH,168.96,62.60,0.871287,2.79,0.386139,5.05,0.625743,28.928507,0.80198,8.344987,0.493069,N/A,N/A
502,ZBRA,542.44,47.35,0.8,12.38,0.833663,6.12,0.694059,32.418297,0.875248,13.697288,0.766337,N/A,N/A
503,ZION,53.42,6.60,0.128713,1.19,0.128713,2.52,0.362376,5.238580,0.057426,2.558296,0.10297,N/A,N/A


# Calculating the Value Score
Value score is the mean of all the percentiles which we calculated

In [24]:
from statistics import mean

for i in real_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(real_df.loc[i, metrics[metric]])
    real_df.loc[i, 'Value Score'] = mean(value_percentiles)

In [25]:
real_df

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Value Score,Number of Shares to Buy
0,A,153.44,51.32,0.823762,9.81,0.79604,8.12,0.817822,31.870553,0.829703,14.969202,0.809901,0.815446,N/A
1,AAL,22.54,-1.51,0.118812,-1.75,0.083168,1.09,0.10297,-3.815853,0.037624,3.181297,0.142574,0.09703,N/A
2,AAP,212.61,23.28,0.429703,4.05,0.530693,1.35,0.161386,11.825002,0.29703,2.826381,0.118812,0.307525,N/A
3,AAPL,143.14,31.53,0.574257,34.29,0.958416,7.20,0.775248,23.126911,0.681188,18.107521,0.881188,0.774059,N/A
4,ABBV,120.26,39.97,0.731683,15.53,0.873267,4.24,0.557426,12.772195,0.348515,8.731831,0.510891,0.604356,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.90,32.14,0.586139,-4.50,0.081188,6.06,0.691089,24.326242,0.710891,10.532166,0.661386,0.546139,N/A
501,ZBH,168.96,62.60,0.871287,2.79,0.386139,5.05,0.625743,28.928507,0.80198,8.344987,0.493069,0.635644,N/A
502,ZBRA,542.44,47.35,0.8,12.38,0.833663,6.12,0.694059,32.418297,0.875248,13.697288,0.766337,0.793861,N/A
503,ZION,53.42,6.60,0.128713,1.19,0.128713,2.52,0.362376,5.238580,0.057426,2.558296,0.10297,0.15604,N/A


# Filtering the top 50 Stocks based on Value Score

In [26]:
top_50_df = real_df.sort_values('Value Score')

In [28]:
top_50_df = top_50_df[:51]

In [30]:
top_50_df.reset_index(drop=True,inplace=True)

In [31]:
top_50_df

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Value Score,Number of Shares to Buy
0,UNM,29.04,7.77,0.132673,0.5634,0.087129,0.4595,0.021782,3.603979,0.047525,0.433195,0.009901,0.059802,N/A
1,MCK,195.80,-7.06,0.09901,-1507.2000,0.00396,0.1317,0.009901,7.574819,0.093069,2.612138,0.10495,0.062178,N/A
2,AIV,7.04,-76.61,0.023762,2.0300,0.29901,-31.7600,0.00198,-44.287605,0.007921,-53.158952,0.005941,0.067723,N/A
3,AIG,48.50,-11.09,0.087129,0.6680,0.091089,0.9643,0.085149,6.093536,0.065347,0.936268,0.025743,0.070891,N/A
4,L,56.64,-409.90,0.00396,0.8770,0.09901,1.0400,0.09901,8.289723,0.134653,1.023003,0.027723,0.072871,N/A
5,HFC,34.20,-35.78,0.047525,1.0200,0.111881,0.4780,0.025743,8.289768,0.136634,2.822101,0.116832,0.087723,N/A
6,AAL,22.54,-1.51,0.118812,-1.7500,0.083168,1.0900,0.10297,-3.815853,0.037624,3.181297,0.142574,0.09703,N/A
7,ABC,117.80,-6.39,0.10297,-87.9600,0.009901,0.1240,0.007921,8.514530,0.152475,4.297614,0.211881,0.09703,N/A
8,HPQ,31.10,11.43,0.184158,-11.2800,0.031683,0.6054,0.041584,8.204995,0.126733,3.272219,0.146535,0.106139,N/A
9,ALL,138.23,12.05,0.192079,1.6500,0.20396,0.8793,0.073267,2.488993,0.041584,0.844410,0.021782,0.106535,N/A


In [32]:
position_size = float(create_portfolio_size()) / len(top_50_df.index)
for i in range(0, len(df['Ticker'])):
    total_share = position_size/top_50_df['Price'][i]
    top_50_df.loc[i,'Number of Shares to Buy'] = math.floor(total_share)

C:\Users\Preet\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [33]:
top_50_df

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Value Score,Number of Shares to Buy
0,UNM,29.04,7.77,0.132673,0.5634,0.087129,0.4595,0.021782,3.603979,0.047525,0.433195,0.009901,0.059802,83358
1,MCK,195.80,-7.06,0.09901,-1507.2000,0.00396,0.1317,0.009901,7.574819,0.093069,2.612138,0.10495,0.062178,12363
2,AIV,7.04,-76.61,0.023762,2.0300,0.29901,-31.7600,0.00198,-44.287605,0.007921,-53.158952,0.005941,0.067723,343852
3,AIG,48.50,-11.09,0.087129,0.6680,0.091089,0.9643,0.085149,6.093536,0.065347,0.936268,0.025743,0.070891,49911
4,L,56.64,-409.90,0.00396,0.8770,0.09901,1.0400,0.09901,8.289723,0.134653,1.023003,0.027723,0.072871,42738
5,HFC,34.20,-35.78,0.047525,1.0200,0.111881,0.4780,0.025743,8.289768,0.136634,2.822101,0.116832,0.087723,70781
6,AAL,22.54,-1.51,0.118812,-1.7500,0.083168,1.0900,0.10297,-3.815853,0.037624,3.181297,0.142574,0.09703,107396
7,ABC,117.80,-6.39,0.10297,-87.9600,0.009901,0.1240,0.007921,8.514530,0.152475,4.297614,0.211881,0.09703,20549
8,HPQ,31.10,11.43,0.184158,-11.2800,0.031683,0.6054,0.041584,8.204995,0.126733,3.272219,0.146535,0.106139,77836
9,ALL,138.23,12.05,0.192079,1.6500,0.20396,0.8793,0.073267,2.488993,0.041584,0.844410,0.021782,0.106535,17512


In [40]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
top_50_df.to_excel(writer, sheet_name='Value Strategy', index = False)

In [41]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [42]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Price-to-Earnings Ratio', float_template],
                    'D': ['PE Percentile', percent_template],
                    'E': ['Price-to-Book Ratio', float_template],
                    'F': ['PB Percentile',percent_template],
                    'G': ['Price-to-Sales Ratio', float_template],
                    'H': ['PS Percentile', percent_template],
                    'I': ['EV/EBITDA', float_template],
                    'J': ['EV/EBITDA Percentile', percent_template],
                    'K': ['EV/GP', float_template],
                    'L': ['EV/GP Percentile', percent_template],
                    'M': ['Value Score', percent_template],
                    'N': ['Number of Shares to Buy', integer_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [43]:
writer.save()